<a href="https://colab.research.google.com/github/mayanku1111/CTR_prediction/blob/main/CTR_deepctr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile
from zipfile import ZipFile
from google.colab import files

files.upload()

!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()

api.authenticate()

download_dir = "/content"

api.dataset_download_files(dataset="gauravduttakiit/clickthrough-rate-prediction", path=download_dir, unzip=True)
!pip install faker

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/gauravduttakiit/clickthrough-rate-prediction
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 48.0 MB/s eta 0:00:00


In [ ]:
import os
import zipfile
from zipfile import ZipFile
from google.colab import files
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from faker import Faker


from deepctr.models import FwFM
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names

data = pd.read_csv('ad_10000records.csv')

data['Timestamp'] = pd.to_datetime(data['Timestamp'])

data['Hour'] = data['Timestamp'].dt.hour
data['Day'] = data['Timestamp'].dt.day
data['Month'] = data['Timestamp'].dt.month

unique_cities = data['City'].unique()
unique_countries = data['Country'].unique()

In [ ]:
label_encoders = {}

for col in ['City', 'Gender', 'Country']:
    label_encoders[col] = LabelEncoder()
    data[col] = label_encoders[col].fit_transform(data[col])

sparse_features = ['City', 'Gender', 'Country', 'Hour', 'Day', 'Month']
dense_features = ['Daily Time Spent on Site', 'Age', 'Area Income', 'Daily Internet Usage']

fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique(), embedding_dim=4)
                          for feat in sparse_features] + [DenseFeat(feat, 1) for feat in dense_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

X = {name: data[name] for name in feature_names}
y = data['Clicked on Ad'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = FwFM(linear_feature_columns, dnn_feature_columns, task='binary')
model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy', 'accuracy'])
history = model.fit(X_train, y_train, batch_size=256, epochs=10, verbose=2, validation_split=0.2)

y_pred = model.predict(X_test, batch_size=256)
y_pred = [1 if pred > 0.5 else 0 for pred in y_pred]
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

class_report = classification_report(y_test, y_pred, output_dict=True)
class_report_df = pd.DataFrame(class_report).transpose()

print("\nClassification Report:")
print(class_report_df)

In [ ]:
fake = Faker()

n_samples = 1000
synthetic_data = pd.DataFrame({
    'Daily Time Spent on Site': np.random.uniform(20, 120, n_samples),
    'Age': np.random.randint(18, 65, n_samples),
    'Area Income': np.random.uniform(15000, 100000, n_samples),
    'Daily Internet Usage': np.random.uniform(50, 300, n_samples),
    'City': [fake.random_element(unique_cities) for _ in range(n_samples)],
    'Gender': [fake.random_element(['Male', 'Female']) for _ in range(n_samples)],
    'Country': [fake.random_element(unique_countries) for _ in range(n_samples)],
    'Timestamp': [fake.date_time_this_year() for _ in range(n_samples)]
})

In [ ]:
synthetic_data['Timestamp'] = pd.to_datetime(synthetic_data['Timestamp'])
synthetic_data['Hour'] = synthetic_data['Timestamp'].dt.hour
synthetic_data['Day'] = synthetic_data['Timestamp'].dt.day
synthetic_data['Month'] = synthetic_data['Timestamp'].dt.month

for col in ['City', 'Gender', 'Country']:
    synthetic_data[col] = label_encoders[col].transform(synthetic_data[col])


synthetic_X = {name: synthetic_data[name] for name in feature_names}

synthetic_pred = model.predict(synthetic_X, batch_size=256)
synthetic_pred = [1 if pred > 0.5 else 0 for pred in synthetic_pred]


np.random.seed(42)
synthetic_labels = np.random.randint(0, 2, n_samples)

accuracy_synthetic = accuracy_score(synthetic_labels, synthetic_pred)
print(f"Accuracy on Synthetic Data: {accuracy_synthetic:.4f}")

In [ ]:
!pip install --upgrade tensorflow

  Using cached h5py-3.12.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.5 kB)
Using cached h5py-3.12.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.3 MB)
  Attempting uninstall: h5py
    Found existing installation: h5py 3.7.0
    Uninstalling h5py-3.7.0:
      Successfully uninstalled h5py-3.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
deepctr 0.9.3 requires h5py==3.7.0; python_version >= "3.9", but you have h5py 3.12.1 which is incompatible.
